# RAG 파이프라인 (Elasticsearch + Sentence‑BERT + OpenAI GPT)
이 노트북은 전체 파이프라인 코드를 **모든 줄 주석**과 함께 보여줍니다.
각 섹션별로 Markdown 설명 → Code 셀 순서로 구성되어 있습니다.

## 📦 라이브러리 임포트 및 모델 초기화
필요 라이브러리 추후 추가

In [ ]:
import os
import json
import jsonlines
import requests
import traceback
import time

from elasticsearch import Elasticsearch, helpers
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv

model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")     # 한국어 SBERT 모델 로드

In [ ]:
# 오류메시지 안나오게 설정
import urllib3

# InsecureRequestWarning 끄기
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

### .env 불러오기

In [ ]:
load_dotenv()

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
ES_PASSWORD=os.getenv("ES_PASSWORD")
SOLAR_API_KEY=os.getenv("SOLAR_API_KEY")
SOLAR_API_URL=os.getenv("SOLAR_API_URL")



## 임베딩 함수
1. get_embedding(sentences) 함수
- sentences: 텍스트(또는 텍스트 리스트)가 들어오는 입력.
- model.encode(sentences): model 객체 (보통 Sentence-BERT 같은 임베딩 모델)을 이용해서 입력 문장들을 벡터로 변환하는 함수.
- 결과적으로 문장 하나당 하나의 벡터를 뽑아주는 역할.

2. get_embeddings_in_batches(docs, batch_size=100) 함수
- docs: 여러 개의 문서를 담은 리스트야. (문서 하나는 딕셔너리 형태, 예: {"content": "내용"})
- batch_size: 한 번에 처리할 문서 수. 기본값은 100개.
- 작동 방식:
    - 전체 문서를 batch_size만큼 나눠서,
    - 각 배치마다 문서의 "content"만 뽑아,
    - get_embedding()으로 임베딩을 계산하고,
    - 결과를 하나의 리스트로 이어붙인다.

매 배치마다 진행 상황을 출력해서, "몇 번째 문서까지 처리했는지" 확인

In [ ]:
def get_embedding(sentences):               # 문장 리스트를 임베딩으로 변환
    return model.encode(sentences)           # 모델을 이용해 문장들을 임베딩 벡터로 변환


def get_embeddings_in_batches(docs, batch_size=100):  # 배치 단위 임베딩 생성
    batch_embeddings = []                              
    for i in range(0, len(docs), batch_size):          # 0부터 끝까지 batch_size씩 슬라이싱
        batch = docs[i:i + batch_size]                 # 현재 배치 추출
        contents = [doc["content"] for doc in batch]   # 문서 본문만 리스트로 추출
        embeddings = get_embedding(contents)           # 임베딩 계산
        batch_embeddings.extend(embeddings)            # 결과 누적
        print(f'batch {i}')                            # 진행 상황 출력
    return batch_embeddings

## ES 인덱스 관리
1. create_es_index(index, settings, mappings)
- index: 만들고 싶은 인덱스 이름 (예: "test")
- settings: 아까 만든 분석기 설정 (settings 변수)
- mappings: 아까 만든 필드 구조 (mappings 변수)

2. delete_es_index(index)
- index: 삭제하고 싶은 인덱스 이름.
- 이름에 해당하는 인덱스를 삭제

3. bulk_add(index, docs)
- index: 데이터를 집어넣을 인덱스 이름.
- docs: 업로드할 문서들 리스트. (doc는 보통 { "content": "텍스트", "embeddings": [...] } 형태)
- 작동 방식:
    - 문서 리스트를 Elasticsearch가 이해할 수 있는 bulk 작업용 액션 리스트로 바꾼다.
    - helpers.bulk를 사용해서 한 번에 대량 업로드한다.

In [ ]:
def create_es_index(index, settings, mappings):        # 새 인덱스 생성 함수
    if es.indices.exists(index=index):                 # 이미 존재하면
        es.indices.delete(index=index)                 # 삭제 후 재생성
    es.indices.create(index=index,                    # 인덱스 생성
                      settings=settings,
                      mappings=mappings)

def delete_es_index(index):                           # 인덱스 삭제 래퍼
    es.indices.delete(index=index)

def bulk_add(index, docs):                            # 대량 색인을 위한 헬퍼
    actions = [{'_index': index, '_source': doc} for doc in docs]  # 액션 목록 생성
    return helpers.bulk(es, actions)                  # helpers.bulk 로 일괄 업로드

## ES 검색함수
1. sparse_retrieve(query_str, size)
- query_str: 검색할 키워드(텍스트).
- size: 몇 개 결과를 가져올지.
- 작동 방식:
    - 사용자가 입력한 query_str을 BM25 방식(기본 텍스트 검색)으로 content 필드에서 매칭시킨다.
    - _score 기준으로 가장 관련성 높은 문서를 size만큼 가져온다

2. dense_retrieve(query_str, size)
- query_str: 검색할 키워드(텍스트).
- size: 몇 개 결과를 가져올지.
- 작동 방식:
    - query_str을 임베딩 벡터로 변환한다.
    - 이 벡터와 인덱스에 저장된 문서들의 벡터(embeddings)를 비교한다.
    - 가장 가까운(유사한) 문서 size개를 가져온다.

In [ ]:
def sparse_retrieve(query_str, size):                 # 역색인(BM25) 검색
    query = {"match": {"content": {"query": query_str}}}
    return es.search(index="test",
                     query=query,
                     size=size,
                     sort="_score")

def dense_retrieve(query_str, size):                  # 벡터 KNN 검색
    query_embedding = get_embedding([query_str])[0]   # 쿼리 임베딩
    knn = {                                           # KNN 파라미터
        "field": "embeddings",
        "query_vector": query_embedding.tolist(),
        "k": size,
        "num_candidates": 100
    }
    return es.search(index="test", knn=knn)           # ES 8.x KNN 검색 호출

def hybrid_retrieve(query_str, size):
    """
    sparse(BM25) 검색 결과 + dense(KNN) 검색 결과를 결합하여 
    더 강력한 검색 결과를 생성하는 hybrid retrieval 함수
    """
    
    # sparse(BM25) 검색 결과 가져오기
    sparse_result = sparse_retrieve(query_str, size)
    
    # dense(KNN) 검색 결과 가져오기
    dense_result = dense_retrieve(query_str, size)
    
    # 문서 ID를 기준으로 결과를 병합하기 위한 딕셔너리 초기화
    merged_hits = {}
    
    # sparse 검색 결과를 먼저 처리
    for hit in sparse_result["hits"]["hits"]:
        doc_id = hit["_id"]  # Elasticsearch 내부 doc id
        merged_hits[doc_id] = {
            "source": hit["_source"],  # 문서 본문
            "sparse_score": hit["_score"],  # sparse(BM25) 스코어
            "dense_score": 0.0  # 초기 dense 스코어는 0으로 설정
        }
    
    # dense 검색 결과를 처리
    for hit in dense_result["hits"]["hits"]:
        doc_id = hit["_id"]  # Elasticsearch 내부 doc id
        if doc_id in merged_hits:
            merged_hits[doc_id]["dense_score"] = hit["_score"]  # 이미 있는 경우 dense 스코어만 추가
        else:
            merged_hits[doc_id] = {
                "source": hit["_source"],  # 문서 본문
                "sparse_score": 0.0,        # sparse 점수는 없는 것으로 간주
                "dense_score": hit["_score"]  # dense 스코어만 있는 문서
            }
    
    # sparse와 dense 스코어를 조합해서 최종 스코어 계산
    for doc_id in merged_hits:
        # sparse와 dense 점수를 가중 평균 (비율은 조정 가능)
        merged_hits[doc_id]["final_score"] = 0.5 * merged_hits[doc_id]["sparse_score"] + 0.5 * merged_hits[doc_id]["dense_score"]
    
    # 최종 스코어 기준으로 정렬 (높은 점수 순)
    ranked_hits = sorted(merged_hits.items(), key=lambda x: x[1]["final_score"], reverse=True)
    
    # top size개만 선택
    top_hits = ranked_hits[:size]
    
    # 출력 형식을 기존 Elasticsearch 검색 결과처럼 맞추기
    results = []
    for doc_id, info in top_hits:
        results.append({
            "_id": doc_id,
            "_score": info["final_score"],
            "_source": info["source"]
        })
    
    return results


## ES 클라이언트 설정
### 서버 버전과 호환이 안됨. 코드 fix 필요

In [ ]:
# es_username = "elastic"                               # ES 기본 사용자
# es_password = "ES_PASSWORD"           # ← 실제 비밀번호로 교체

# es = Elasticsearch(                                   # 클라이언트 생성
#     ['https://localhost:9200'],
#     basic_auth=(es_username, es_password),
#     ca_certs="./elasticsearch-8.8.0/config/certs/http_ca.crt"
# )
# print(es.info())                                      # 접속 정보 출력

### 호환 모드

In [ ]:
es_username = "elastic"                               # ES 기본 사용자
es_password = ES_PASSWORD           # ← 실제 비밀번호로 교체

es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=(es_username, es_password),
    verify_certs=False,        # CA 무시(테스트용)
)

print(es.info())  # 이제 정상 출력 (예: 7.17.9)


## 인덱스 설정 & 매핑
Elasticsearch에서 한국어 텍스트 + 임베딩 벡터를 저장할 수 있는 인덱스를 만들기 위한 설정.  

1. settings 부분
- nori 분석기는 Nori Tokenizer를 기반으로 함.
- 조사, 어미, 기호 같은 불필요한 품사 토큰을 제거
    - E(어미), J(조사), SC(구두점), SE(문장구분), SF(마침표), VCN(형용사), VCP(긍정지정사), VX(보조동사)
- 핵심 던어만 남김.

2. mappings 부분
- content: 일반 텍스트 필드, 검색할 때 nori 분석기로 전처리 해서 인덱싱.
- embeddings: 768 차원 벡터 필드를 저장하고, 벡터 검색을 할 수 있도록 설정.

In [ ]:
settings = {                                          # 한글용 Nori 분석기 설정
    "analysis": {
        "analyzer": {
            "nori": {                               # 'nori'라는 사용자 정의 분석기를 만든다
                "type": "custom",                   # 직접 필터와 토크나이저를 설정하는 방식
                "tokenizer": "nori_tokenizer",      # 한국어용 기본 Nori 토크나이저 사용
                "decompound_mode": "mixed",         # 복합어는 분리도 하고 원형도 같이 보존(mixed)
                "filter": ["nori_posfilter", "synonym_filter"]        # 필터링 적용
            }
        },
        
        "normalizer": {
            "lowercase_normalizer": {
            "type": "custom",
            "filter": ["lowercase"]
            }
        },

        "filter": {
            "synonym_filter": {
                "type": "synonym",
                # "synonyms_path": "/usr/share/elasticsearch/config/synonym_filter.txt", # 파일 경로 # 헬륨, helium, HeLiUm
                "synonyms": ["운동, 체육, 스포츠"] # 헬륨, helium, HeLiUm
                # "updateable": True  #  (7.3 이상) 동적 업데이트
            },
            "nori_posfilter": {  # 사용자 정의 품사 필터
                "type": "nori_part_of_speech",  # 품사 기반 필터링을 수행
                "stoptags": ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX"]  
                # 이 품사에 해당하는 토큰들은 제거한다
                # E(어미), J(조사), SC(구두점), SE(문장구분), SF(마침표), VCN(형용사), VCP(긍정지정사), VX(보조동사)
            }
        }
    }
}

mappings = {
    "properties": {
        "content": {"type": "text", "analyzer": "nori"},  # 'content' 필드는 위에서 정의한 nori 분석기로 분석
        "embeddings": {  
            "type": "dense_vector",  # 'embeddings' 필드는 밀집 벡터(dense vector)로 저장
            "dims": 768,  # 임베딩 벡터 차원 수는 768 (ex: BERT base 모델 output 차원)
            "index": True,  # 벡터를 검색(indexing) 가능하게 설정
            "similarity": "l2_norm"  # 벡터 유사도는 L2 노름(유클리드 거리) 기반으로 측정
        }
    }
}

create_es_index("test", settings, mappings)           # ‘test’ 인덱스 생성


## 문서 로드 & 색인

In [ ]:
index_docs = []                                       # 색인용 문서 리스트
with jsonlines.open("./data/documents.jsonl") as reader:
    docs = list(reader)                # 각 줄이 dict로 바로 변환됨

embeddings = get_embeddings_in_batches(docs)          # 임베딩 배치 생성

for doc, emb in zip(docs, embeddings):                # 문서와 임베딩 병합
    doc["embeddings"] = emb.tolist()                  # numpy → list 변환
    index_docs.append(doc)

ret = bulk_add("test", index_docs)                    # ES에 대량 색인
print(ret)                                            # 색인 결과 출력

## 검색 예시 실행 

In [ ]:
test_query = "금성이 다른 행성들보다 밝게 보이는 이유는 무엇인가요?"  # 샘플 쿼리

search_result_retrieve = sparse_retrieve(test_query, 3)  # BM25 검색
for hit in search_result_retrieve['hits']['hits']:       # 결과 출력
    print('sparse score:', hit['_score'],
          'source:', hit['_source']["content"])

search_result_retrieve = dense_retrieve(test_query, 3)   # 벡터 검색
for hit in search_result_retrieve['hits']['hits']:
    print('dense score:', hit['_score'],
          'source:', hit['_source']["content"])
    
search_result_retrieve = hybrid_retrieve(test_query, 3)   # 하이브리드 검색
for hit in search_result_retrieve:
    print('hybrid score:', hit['_score'],
          'source:', hit['_source']["content"])

## OpenAI RAG 설정

In [ ]:
from openai import OpenAI                             # OpenAI SDK
import traceback                                      # 예외 트레이스 출력용

os.environ["SOLAR_API_KEY"] = SOLAR_API_KEY         # API 키 환경변수
client = OpenAI(                                    # OPENAI API 클라이언트 생성
    base_url="https://api.upstage.ai/v1/solar",     # Upstage Solar API URL
    api_key=SOLAR_API_KEY
)
llm_model = "solar-pro"                             # 사용할 LLM 이름

persona_qa = """                                      # QA용 프롬프트
## Role: 검색 기반 과학 상식 답변 전문가

## Instructions
- 제공된 검색 문서(Reference)와 사용자의 메시지를 기반으로 답변을 생성하세요.
- 반드시 검색 문서에 기반하여 답변해야 하며, 문서에 없으면 "정보 부족으로 답변할 수 없습니다."라고 답변하세요.
- 답변은 간결하고 명확하게 작성하세요. (3문장 이내)
- 답변은 반드시 한국어로 작성하세요.
- 검색 문서가 여러 개일 경우, 가장 관련성 높은 내용을 우선 사용하세요.
"""

persona_function_calling = """
## Role: 과학 상식 관련 질문 필터링 전문가

## Instructions
- 사용자의 질문이 과학 상식(자연 과학, 생명 과학, 물리학, 화학, 지구 과학 등)과 관련되어 있으면 search API를 호출하세요.
- 과학 상식 외(일상 대화, 개인 감정, 문학, 철학, 사회 이슈 등) 질문이라면 직접 답변을 생성하세요.
- 과학 관련 여부를 판단할 수 없는 경우에는 search API를 호출하지 않고, "질문을 명확히 해주세요"라고 답변하세요.
"""

tools = [                                             # function‑calling 정의
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "과학 상식과 관련된 질문에 대해 적합한 문서를 검색합니다",
            "parameters": {
                "type": "object",
                "properties": {
                    "standalone_query": {
                        "type": "string",
                        "description": "과학 상식 관련 한글 질문을 명확하게 입력하세요"

                    }
                },
                "required": ["standalone_query"]
            }
        }
    }
]

### SOLAR CALL

In [ ]:
def call_solar(messages, model="solar-1-mini-chat", temperature=0.0, top_p=0.9):
    headers = {
        "Authorization": f"Bearer {SOLAR_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "top_p": top_p,
        "n": 1
    }
    
    response = requests.post(SOLAR_API_URL, headers=headers, json=payload)

    if response.status_code == 200:
        return response.json()  # ✅ 여기! .text 말고 .json()으로!!
    else:
        print(f"Solar API Error {response.status_code}: {response.text}")
        raise Exception(f"Solar API 호출 실패 (status_code: {response.status_code})")


## RAG 핵심 로직
### open_ai 이용

In [ ]:
def answer_question(messages):  # 대화 기록(messages)을 입력 받아 답변(response)을 생성하는 함수
    response = {  # 반환할 결과를 저장할 빈 response 딕셔너리 초기화
        "standalone_query": "",  # 검색용 최종 쿼리 저장
        "topk": [],              # 검색된 문서들의 docid 리스트 저장
        "references": [],        # 검색된 문서들의 score와 내용 저장
        "answer": ""             # 최종 생성된 답변 저장
    }

    msg = [{"role": "system", "content": persona_function_calling}] + messages  # 검색 여부를 판단할 system 프롬프트와 유저 메시지를 합친다
    try:  # 1차 GPT 호출을 시도
        result = client.chat.completions.create(  # OpenAI 클라이언트를 통해 LLM 호출
            model=llm_model,                      # 사용할 모델 이름 설정
            messages=msg,                         # 입력 메시지 설정
            tools=tools,                          # function-calling용 tool 설정
            temperature=0,                        # 랜덤성 최소화 (0으로 설정)
            seed=1,                               # 결과 재현성을 위해 시드 고정
            timeout=10                            # API 호출 제한 시간 설정 (초 단위)
        )
    except Exception:  # 예외가 발생했을 때
        traceback.print_exc()  # 에러 로그를 출력
        response["answer"] = "답변을 생성하는 도중 오류가 발생했습니다."  # 오류 메시지를 답변에 설정
        return response  # 오류 발생 시 빈 response 반환

    if result.choices[0].message.tool_calls:  # tool_call이 존재하면 → 검색이 필요한 경우
        tool_call = result.choices[0].message.tool_calls[0]  # 첫 번째 tool_call을 가져온다
        args = json.loads(tool_call.function.arguments)  # tool_call 안의 arguments를 파싱해서 딕셔너리로 변환
        query = args.get("standalone_query")  # 검색용 쿼리를 가져온다

        search_result = sparse_retrieve(query, 3)  # BM25 기반 검색을 수행하여 관련 문서 3개를 찾는다
        response["standalone_query"] = query  # response에 검색 쿼리를 기록한다

        context = []  # 검색된 문서들의 본문을 담을 리스트 초기화
        for hit in search_result['hits']['hits']:  # 검색 결과 문서들을 하나씩 순회
            context.append(hit["_source"]["content"])  # 문서 내용을 context 리스트에 추가
            response["topk"].append(hit["_source"]["docid"])  # 문서의 docid를 topk 리스트에 추가
            response["references"].append({  # 문서의 score와 content를 references 리스트에 추가
                "score": hit["_score"], 
                "content": hit["_source"]["content"]
            })

        messages.append({  # 검색 결과를 Assistant 메시지로 추가
            "role": "assistant",
            "content": json.dumps(context)  # 검색 문서 내용을 JSON 문자열로 변환하여 삽입
        })
        qa_msg = [{"role": "system", "content": persona_qa}] + messages  # QA 생성용 system 프롬프트와 메시지들을 합친다

        try:  # 2차 GPT 호출을 시도 (최종 답변 생성)
            qa_res = client.chat.completions.create(
                model=llm_model,              # 사용할 모델 설정
                messages=qa_msg,               # 검색 문서를 포함한 메시지 전달
                temperature=0,                 # 랜덤성 최소화
                seed=1,                        # 결과 재현성 확보
                timeout=30                     # API 호출 제한 시간 설정
            )
        except Exception:  # 예외 발생 시
            traceback.print_exc()  # 에러 로그를 출력
            return response  # 오류 발생 시 현재까지의 response 반환

        response["answer"] = qa_res.choices[0].message.content  # 최종 생성된 답변을 response에 저장
    else:  # tool_call이 없으면 → 검색 없이 바로 답변
        response["answer"] = result.choices[0].message.content  # 1차 호출 결과로 바로 답변 설정

    return response  # 최종 생성된 response 반환


### SOLAR_API 이용

In [ ]:
def answer_question(messages):
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    # 1차 호출 (검색 여부 판단)
    msg = [{"role": "system", "content": persona_function_calling}] + messages
    try:
        result_json = call_solar(msg, model=llm_model, temperature=0.0)  # 이미 dict로 반환됨
    except Exception:
        traceback.print_exc()
        response["answer"] = "답변을 생성하는 도중 오류가 발생했습니다."
        return response

    message = result_json["choices"][0]["message"]  # ✅ Solar 결과 접근
    tool_calls = message.get("tool_calls", None)    # tool_calls 있는지 확인

    # tool_calls가 있으면 검색 필요
    if tool_calls:
        tool_call = tool_calls[0]
        args = json.loads(tool_call["function"]["arguments"])
        query = args.get("standalone_query")

        search_result = sparse_retrieve(query, 3)
        response["standalone_query"] = query

        context = []
        for hit in search_result['hits']['hits']:
            context.append(hit["_source"]["content"])
            response["topk"].append(hit["_source"]["docid"])
            response["references"].append({
                "score": hit["_score"],
                "content": hit["_source"]["content"]
            })

        messages.append({
            "role": "assistant",
            "content": json.dumps(context)
        })
        qa_msg = [{"role": "system", "content": persona_qa}] + messages

        # 2차 호출 (최종 답변 생성)
        try:
            qa_result_json = call_solar(qa_msg, model=llm_model, temperature=0.0)
        except Exception:
            traceback.print_exc()
            return response

        response["answer"] = qa_result_json["choices"][0]["message"]["content"]  # ✅ 최종 답변 뽑기
    else:
        # 검색 없이 바로 답변
        response["answer"] = message["content"]

    return response


## 평가 루프 
### OPENAI

In [ ]:
def eval_rag(eval_filename, output_filename):  # 평가를 수행하고 결과를 저장하는 함수 정의
    with jsonlines.open(eval_filename) as reader, open(output_filename, "w") as of:  
        # 입력 파일(eval_filename)을 읽기 모드로 열고, 출력 파일(output_filename)을 쓰기 모드로 연다
        for idx, j in enumerate(reader):  # 입력 파일에서 한 줄씩 읽어오고, 인덱스(idx)와 내용(j)을 가져온다
            print(f'Test {idx}\nQuestion: {j["msg"]}')  # 현재 테스트 번호와 질문 내용을 출력한다
            
            resp = answer_question(j["msg"])  # 현재 질문(j["msg"])에 대해 RAG 시스템을 실행하여 응답(resp)을 생성한다
            
            print(f'Answer: {resp["answer"]}\n')  # 생성된 답변을 출력한다

            out = {  # 제출 파일에 쓸 결과를 딕셔너리 형태로 정리한다
                "eval_id": j["eval_id"],  # 평가 아이디 (문제 고유 식별자)
                "standalone_query": resp["standalone_query"],  # 생성된 검색용 최종 쿼리
                "topk": resp["topk"],  # 검색해서 가져온 문서들의 docid 리스트
                "answer": resp["answer"],  # 최종 생성된 답변 텍스트
                "references": resp["references"]  # 검색된 문서들의 스코어 및 내용 리스트
            }
            
            of.write(json.dumps(out, ensure_ascii=False) + "\n")  
            # 결과 딕셔너리(out)를 JSON 문자열로 변환해 출력 파일에 한 줄씩 저장한다

            time.sleep(2.0)  # ✅ 요청 사이 딜레이 조금 여유있게 (2초)


# 실행
eval_rag("./data/eval.jsonl", "sample_submission.csv")  # 위에서 정의한 eval_rag 함수를 호출하여 평가를 시작한다


### SOLAR_API

In [ ]:
import requests


def eval_rag(eval_filename, output_filename):
    with jsonlines.open(eval_filename) as reader, open(output_filename, "w") as of:
        for idx, j in enumerate(reader):
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            
            try:
                resp = answer_question(j["msg"])  # Solar API를 사용하는 answer_question 호출
            except Exception as e:
                print(f"Error during answering: {e}")
                resp = {"standalone_query": "", "topk": [], "references": [], "answer": "답변 생성 실패"}

            print(f'Answer: {resp["answer"]}\n')

            out = {
                "eval_id": j["eval_id"],
                "standalone_query": resp["standalone_query"],
                "topk": resp["topk"],
                "answer": resp["answer"],
                "references": resp["references"]
            }
            of.write(json.dumps(out, ensure_ascii=False) + "\n")

            time.sleep(1.5)  # ✅ 요청 간 딜레이 추가

eval_rag("./data/eval1.jsonl", "sample_submission2.csv")  # 위에서 정의한 eval_rag 함수를 호출하여 평가를 시작한다

